<a href="https://colab.research.google.com/github/sashabel1/Cloud-tirgul/blob/master/cloudSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4
!pip install firebase

In [ ]:
DBLink = "https://alexaanddaniel-c1413-default-rtdb.firebaseio.com/"

In [ ]:
url = 'https://mqtt.org/'

In [5]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from nltk.stem import PorterStemmer
from firebase import firebase

# קריאה לדף אינטרנט
def fetch_page(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
    except:
        return None
    return None

# חילוץ מילים
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        index[word] = index.get(word, 0) + 1
    return index

# הסרת מילים חסרות משמעות
def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
    return {word: count for word, count in index.items() if word not in stop_words}

# Stemming
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed = stemmer.stem(word)
        stemmed_index[stemmed] = stemmed_index.get(stemmed, 0) + count
    return stemmed_index

# בניית אינדקס הופכי (מילה -> רשימת דפים)
def add_to_inverted_index(inverted_index, index, doc_id):
    for word in index:
        if word not in inverted_index:
            inverted_index[word] = []
        if doc_id not in inverted_index[word]:
            inverted_index[word].append(doc_id)

# חילוץ קישורים פנימיים
def get_internal_links(soup, base_url, domain):
    links = set()
    for link_tag in soup.find_all('a', href=True):
        href = link_tag['href']
        full_url = urljoin(base_url, href)
        parsed = urlparse(full_url)
        if parsed.netloc == domain:
            cleaned_url = parsed.scheme + "://" + parsed.netloc + parsed.path
            links.add(cleaned_url)
    return links

# סריקה לעומק של האתר
def crawl_website(start_url):
    visited = set()
    to_visit = [start_url]
    domain = urlparse(start_url).netloc
    inverted_index = {}

    while to_visit:
        current_url = to_visit.pop()
        if current_url in visited:
            continue

        print(f"Scanning: {current_url}")
        soup = fetch_page(current_url)
        if soup is None:
            continue

        visited.add(current_url)

        # עיבוד המילים בדף
        index = index_words(soup)
        index = remove_stop_words(index)
        index = apply_stemming(index)
        add_to_inverted_index(inverted_index, index, current_url)

        # מציאת קישורים פנימיים
        internal_links = get_internal_links(soup, current_url, domain)
        for link in internal_links:
            if link not in visited:
                to_visit.append(link)

    return inverted_index

# כתובת למסד הנתונים
DBLink = "https://alexaanddaniel-c1413-default-rtdb.firebaseio.com/"
firebase_conn = firebase.FirebaseApplication(DBLink, None)

# URL התחלתי
start_url = 'https://mqtt.org/'

# בניית האינדקס
inverted_index = crawl_website(start_url)

# הדפסת האינדקס למסך
print("\n=== Inverted Index ===")
for word, doc_list in inverted_index.items():
    print(f"{word} -> {doc_list}")

# העלאה ל-Firebase
firebase_conn.put('/', 'inverted_index', inverted_index)
print("\nהאינדקס הועלה בהצלחה ל-Firebase.")


Streaming output truncated to the last 5000 lines.
v5t -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
zw5t -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
ҹq -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
armi -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
wϻk -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
qql -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
ǻhm -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
7qda -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
9k3 -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
ȉ6 -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
hzn -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
sɵ -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
ߙɣ -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
壗 -> ['https://mqtt.org/assets/downloads/mqtt-logo-assets.zip']
rr6 -> ['https://mqtt.org/as